# Build a ChatBot

*Taken from the [Langchain documentation](https://python.langchain.com/v0.2/docs/tutorials/chatbot/)*

## Overview

We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- [Conversational RAG](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/): Enable a chatbot experience over an external source of data
- [Agents](https://python.langchain.com/v0.2/docs/tutorials/agents/): Build a chatbot that can take actions

This tutorial will cover the basics which will be helpful for those two more advanced topics, but feel free to skip directly there should you choose.

## Concepts

Here are a few of the high level components we'll be working with:

- `Chat Models`: The chatbot interface is based around messages rather than raw text, and therefore is best suited to Chat MOdels rather than text LLMs.
- `Prompt Templates`, which simplify the process of assembling prompts that combine default messages, user input, chat history, and (optionally) addional retrieved context.
- `Chat History`, which allows a chatbot to "remember" past interactions and take them into account when responding to followup questions.

We'll cover how to fit the above components together to create a powerful conversational chatbot.

## Setup

### Jupyter Notebook

We're assumed to be doing this in a Jupyter Notebook.

### Installation

To install LangChain run:

```ps
conda install langchain -c conda-forge
```
### LangSmith

We don't p2w. We will not be using LangSmith. If we need debugging on the inference chain we'll try langserve.

### Quickstart

Review of how to use the LangChain API:

In [3]:
from langchain_community.chat_models import ChatOllama

model = ChatOllama(model="llama3:instruct", base_url="http://localhost:11434")

Let's first use the model directly. `ChatModel`s are instances of LangChain "Runnables", which means they expose a standard interface for interacting with them. To just simply call the model, we can pass in a list of messages to the `.invoke` method.

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

AIMessage(content="Hi Bob! It's nice to meet you. How can I help or chat with you today?", response_metadata={'model': 'llama3:instruct', 'created_at': '2024-05-27T22:36:54.5520683Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5287063800, 'load_duration': 4824702900, 'prompt_eval_count': 14, 'prompt_eval_duration': 86566000, 'eval_count': 21, 'eval_duration': 374764000}, id='run-b2fd6514-4f45-4b05-8aff-e31819bf8d58-0')

The model on its own does not have any concept of state. For example, if you ask a followup question:

In [5]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I'm happy to help! However, I don't think we've had a chance to introduce ourselves yet. Since this is the beginning of our conversation, I don't have any information about your name. Would you like to tell me what it is?", response_metadata={'model': 'llama3:instruct', 'created_at': '2024-05-27T22:38:27.9432492Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 1330133500, 'load_duration': 996800, 'prompt_eval_count': 10, 'prompt_eval_duration': 339204000, 'eval_count': 52, 'eval_duration': 988567000}, id='run-333bd400-aa65-46f3-aee6-4aa3194e7618-0')

We can see that it doesn't take the previous conversation turn into context, and cannot answer the question. This makes for a terrible chatbot experience! To get around this, we need to pass the entire conversation history into the model. Let's see what happens when we do that:

In [7]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Bob"),
        AIMessage(content="Hi Bob! It's nice to meet you. How can I help or chat with you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Bob, right?', response_metadata={'model': 'llama3:instruct', 'created_at': '2024-05-27T22:47:37.9952438Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 5151815200, 'load_duration': 4860013200, 'prompt_eval_count': 49, 'prompt_eval_duration': 154827000, 'eval_count': 8, 'eval_duration': 135484000}, id='run-ea8d466e-cec2-4e16-97f1-d2fef3ce6746-0')

And now we can see that we get a good response! 

This is the basic idea underpinning a chatbot's ability to interact conversationally. So how do we best implement this?

## Message History

We can use a message history class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this! 

First, let's make sure to install `langchain-community`, as we will be using an integration in there to store message history.

```ps
pip install langchain-community
```

After that, we can import the relevant classes and set up our chain which wraps the model and adds in this message history. A key part here is the function we pass into as the `get_session_history`. This function is expected to take in a `session_id` and return a Message History object. This `session_id` is used to distinguish between separate conversations, and should be passed in as part of the config when calling the new chain (we'll show how to do that).

In [8]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

We now need to create a `config` that we pass into the runnable every time. This config contains information that is not part of the input directly, but is still useful. In this case, we want to include a `session_id`. This should look like:

In [9]:
config = {"configurable": {"session_id": "abc2"}}

In [10]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Bob")],
    config=config,
)

response.content

Parent run a0b89b7b-0e53-4297-8531-c2b62de920cd not found for run 0fe040f4-6637-4a7c-b840-7e9f3a759af1. Treating as a root run.


"Hi Bob! It's nice to meet you. How's your day going so far?"

In [11]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run f47ba5b6-462a-4fa6-959d-de2109dbb622 not found for run 48e5def6-8a45-4dc9-9f6b-463a7ab20c74. Treating as a root run.


'I remember! Your name is Bob, right?'

Our chatbot now remembers things about us. If we change the config to reference a different `session_id`, we can see that it starts the conversation fresh.

In [12]:
config = {"configurable": {"session_id": "abc3"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 0f712f2d-2bf7-4251-bef6-2affbfcfeb81 not found for run 2f18414e-7de7-4461-a85a-085d86dc891a. Treating as a root run.


"I apologize, but I don't have any information about your name. As a conversational AI, I'm designed to interact with many users without retaining personal information. Each time you interact with me, it's like starting a new conversation. Would you like to introduce yourself and start fresh?"

Since we're persisting the original conversation in a database we can go back:

In [13]:
config = {"configurable": {"session_id": "abc2"}}

response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run 70e65d8c-1bb6-48d3-bb52-a8612b3f911a not found for run 54f4f9d9-9412-43af-99fc-114bd035bb8b. Treating as a root run.


"Your name is Bob, isn't it?"

This is how we can support a chatbot having conversations with many users. Right now all we've done is added a simple persistence layer around the model. We can start to make this more complicated and personalized by adding a prompt template.

## Prompt Templates

Prompt templates help to turn raw user information into a format that the LLM can work with. In this case the raw user input is just a message which we are passint to the LLM. We can complicate this by adding a system message with some custom instructions. To do this we will create a `ChatPromptTemplate`. We will utilize `MessagesPlaceholder` to pass all the messages in.

In [16]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

Note: this slightly changes the input type -- rather than pass in a list of messages we are now passing in a dictionary with a `messages` key where that contains a list of messages.

In [17]:
response = chain.invoke({"messages": [HumanMessage(content="Hi! I'm Bob")]})

response.content

"Hi Bob! It's great to meet you! What brings you here today? Do you have any questions or topics you'd like to chat about? I'm here to help and listen!"

This enables us to wrap it in the same `MessageHistory` object as before:

In [19]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)

config = {"configurable": {"session_id": "abc5"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi! I'm Jim")],
    config=config,
)

response.content

Parent run 51cc4aff-b47b-48e2-94c0-32ef40ff3449 not found for run 6b0b5edd-ed67-4250-b72f-c4a40d08695e. Treating as a root run.


"Nice to meet you, Jim! How's your day going so far? Do you have any specific topics or questions on your mind that you'd like to chat about? I'm here to help with anything you need!"

In [20]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

Parent run bed95a8a-3c36-4a80-b4be-b3bf77b2e9f7 not found for run 5d426483-219d-40ab-b4cd-9d52eccf1bed. Treating as a root run.


'You told me earlier, Jim! Your name is Jim.'

Now let's add multi-lingual support to the chatbot. We can do this by adding a `language` key in the system message and including the key:value pair when we call the `invoke` method.

In [22]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

response = chain.invoke(
    {"messages": [HumanMessage(content="Hi! I'm Bob")], "language": "Spanish"}
)

response.content

'Hola! ¡Bienvenido, Bob! ¿En qué puedo ayudarte hoy? (Hello! Welcome, Bob! How can I help you today?)'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [23]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc11"}}

response = with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi! I'm Todd")], "language": "Spanish"},
    config=config,
)

response.content

Parent run c970a340-c928-4798-983d-6154a7e0171c not found for run bf776edc-9f74-4446-bb02-921757cc3f1d. Treating as a root run.


'Hola Todd! ¡Bienvenido! (Hello Todd! Welcome!) ¿En qué puedo ayudarte hoy? (How can I help you today?)'

In [24]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="What's my name?")], "language": "Spanish"},
    config=config,
)

response.content

Parent run b70b81d7-7575-4a9e-a3a4-63a976b21fdd not found for run d0554931-950b-449a-9f76-71a04156861e. Treating as a root run.


'¡Tu nombre es Todd! (Your name is Todd!)'

## Managing Conversation History

One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged the list of messages will grow unbounded and overflow the context window. Therefore, it is important to add a step that limits the size of the messages you are passing in.

Importantly, you will want to do this before the prompt template but after you load previous messages from `MessageHistory`.

We can do this by adding a simple step in front of the prompt that modifies the `messages` key appropriately and then wrap that new chain in the `MessageHistory` class. First let's define a function that will modify the messages passed in. Let's make it so that it selects the `k` most recent messages. We can then create a new chain by adding it at the start.

In [25]:
from langchain_core.runnables import RunnablePassthrough

def filter_messages(messages, k=10):
    return messages[-k:]

chain = (
    RunnablePassthrough.assign(messages=lambda x: filter_messages(x["messages"]))
    | prompt
    | model
)

If we create a list of messages more than 10 messages long we can see that it forgets the oldest messages.

In [30]:
messages = [
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like moose tracks ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What's my name?")],
        "language": "English",
    }
)

response.content

"I don't think we established a specific name for you earlier, so I don't have one to share!"

We can wrap the chain in a `MessageHistory` object to persist the conversation history.

In [31]:
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)

config = {"configurable": {"session_id": "abc20"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What's my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run e9860e72-3d3c-4fa5-b519-4d2a3c667e78 not found for run 4cc2edbe-7783-4bfa-8b3d-0fadabf60982. Treating as a root run.


"I don't think you mentioned your name earlier, but I'm happy to chat with you regardless!"

Now we've added to the chat history, this means even more information is inaccessible.

In [32]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What's my favorite ice cream?")],
        "language": "English",
    },
    config=config,
)

response.content

Parent run a7e54672-d005-44d9-a39f-508d96f2b9b1 not found for run 048de97e-cf14-4471-8c5d-fb710b2afedf. Treating as a root run.


'Moose Tracks ice cream, right?'

*Hmm... whoever wrote this can't count. Regardless the old outputs are turnicated and it shouldn't know the answer to the question if we lower the `k` value.*

## Streaming

This takes us most of the way to a chatbot. However one important user experience consideration for chatbot application is streaming. LLMs can take a while to respond, in order to improve the UX one thing that most applications do is stream back each token as it is generated. This allows us to see progress! 

All chains expose a `.stream` method and ones that use message history are no different. We can call the method to get back a streaming response.

In [34]:
config = {"configurable": {"session_id": "abc15"}}

for r in with_message_history.stream(
    {
        "messages": [HumanMessage(content="Hi! I'm Todd. Please tell me a joke")],
        "language": "English",
    },
    config=config,
):
    print(r.content, end="|")

Parent run c3f6ff78-d767-4c13-8963-1382a1bd153f not found for run 266e5300-abde-4c79-a4e9-0be16c0c2002. Treating as a root run.


Hey| Todd|!| Here|'s| one|:

|Why| couldn|'t| the| bicycle| stand| up| by| itself|?

|(W|ait| for| it|...)

|Because| it| was| two|-t|ired|!

|Hope| that| made| you| laugh|,| buddy|!||

Next up is [Conversational RAG](https://python.langchain.com/v0.2/docs/tutorials/qa_chat_history/) which is a chatbot that has an external source of data. This may be a useful way turn the conversation history into a database that we can query against.